In [16]:
import sys  
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')


In [17]:
from datetime import timedelta, date
import pickle
from ruffus import *
import os
import xarray as xr
import pandas as pd
import numpy as np
import re
import dataconfig

In [22]:
WORKING_DIR = dataconfig.DATA_DIR_GOES_FLUX

DATA_PRODUCT_DIR = dataconfig.DATA_DIR_PRODUCTS

tw = lambda x: os.path.join(WORKING_DIR, x)



###### REVIEW AND OPTIMIZE Jan17,2020 ###########


# some dictionaries we'll use sometimes
instruments = ['goes13', 'goes14', 'goes15']

inst_dict = {'goes13': 'g13', 'goes14': 'g14', 'goes15': 'g15'}

ftp_prename = 'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs'
# ftp_prename = 'https://ncei.noaa.gov/data/goes-space-environment-monitor/access/science/xrs'


#### set the date limits of the data you want to download

# start_download_from = input('Insert date and time you would like your XRay Flux Data to START from. For example: YYYY-MM-DD HH:MM:SS ' )

# end_download_at = input('Insert date and time you would like your XRay Flux Data to END at. For example: YYYY-MM-DD HH:MM:SS ' )

start_download_from = '2010-05-01T00:00:00'
end_download_at = '2010-05-02T00:00:00'

START_DATE_TIME = pd.Timestamp(start_download_from, tz = 'utc')

END_DATE_TIME = pd.Timestamp(end_download_at, tz = 'utc')


def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)


all_dates_list = []

# start_dt = date(2010, 5, 1)
# end_dt = date(2020,3,5)
# for dt in daterange(start_dt, end_dt):
#     # all_dates_list.append(dt.strftime("%Y%m%d"))
#     all_dates_list.append(dt)


for dt in daterange(START_DATE_TIME, END_DATE_TIME):
    # all_dates_list.append(dt.strftime("%Y%m%d"))
    all_dates_list.append(dt)


ftp_name_dict = []

for full_instrument_name in instruments:

    for this_date in all_dates_list:

        specific_date_str = this_date.strftime("%Y%m%d")

        year, month, day = this_date.strftime('%Y'), this_date.strftime('%m'), this_date.strftime('%d')

        specific_file_name = f'sci_gxrs-l2-irrad_{inst_dict[full_instrument_name]}_d{specific_date_str}_v0-0-0.nc'

        ftp_name = f'{ftp_prename}/{full_instrument_name}/gxrs-l2-irrad_science/{year}/{month}/{specific_file_name}'

        outfile_str = tw(f'{year}-{month}-{day}_{full_instrument_name}_irrad.nc')

        wget_log_file = f'{outfile_str}.wget_log'

        ftp_name_dict.append({'download_file': ftp_name, 'out_name': outfile_str, 'wget_log_file': wget_log_file })

In [23]:
queries = ftp_name_dict
queries

[{'download_file': 'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes13/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g13_d20100501_v0-0-0.nc',
  'out_name': '/mnt/e/image_data/xrs_data_flux/2010-05-01_goes13_irrad.nc',
  'wget_log_file': '/mnt/e/image_data/xrs_data_flux/2010-05-01_goes13_irrad.nc.wget_log'},
 {'download_file': 'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes13/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g13_d20100502_v0-0-0.nc',
  'out_name': '/mnt/e/image_data/xrs_data_flux/2010-05-02_goes13_irrad.nc',
  'wget_log_file': '/mnt/e/image_data/xrs_data_flux/2010-05-02_goes13_irrad.nc.wget_log'},
 {'download_file': 'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes14/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g14_d20100501_v0-0-0.nc',
  'out_name': '/mnt/e/image_data/xrs_data_flux/2010-05-01_goes14_irrad.nc',
  'wget_log_file': '/mnt/e/image_data/xrs_data_flux/2010-05-01_goes14_irrad.nc.wget_log'},
 {'download_file': 'ftp://satdat.ngdc

In [27]:
test_query = queries[0]
ftp_query_name = test_query['download_file']
outfile_name = test_query['out_name']
wget_log_file = test_query['wget_log_file']

#     ############### EDIT ###################

#         # Include "outname" in oufile dict in order to offer continuity to the pipeline

# # ftp_query_name = 'https://ncei.noaa.gov/data/goes-space-environment-monitor/access/science/xrs/goes14/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g14_d20100501_v0-0-0.nc'
# outfile_name = queries['out_name']

# # print(outfile)

download_str = f'wget -e robots=off --recursive --no-parent -A --directory-prefix {WORKING_DIR} --no-directories --verbose False {ftp_query_name} -O {outfile_name} -o {wget_log_file}' 

os.system(download_str)
# print(download_str)

1024

In [21]:
# ftp_query_name = 'https://ncei.noaa.gov/data/goes-space-environment-monitor/access/science/xrs/goes14/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g14_d20100501_v0-0-0.nc'
# ftp_query_name = 'https://ncei.noaa.gov/data/goes-space-environment-monitor/access/science/xrs/goes14/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g14_d20100501_v0-0-0.nc'
# ftp_prename = 'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs'

# for element in queries:
#     print(element[''])

    # ftp_query_name = element['download_file']
    # print(ftp_query_name)
    # response = os.system(f'wget -q {ftp_query_name}')
    # print(response)

KeyError: ''

In [ ]:
ftp_query_name

'ftp://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes14/gxrs-l2-irrad_science/2010/05/sci_gxrs-l2-irrad_g14_d20100501_v0-0-0.nc'

In [28]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

In [29]:
parent_dir

'/home/padialjr/ALEXIS_pipeline'

In [30]:
current_dir

'/home/padialjr/ALEXIS_pipeline/jupyter_notebooks'

In [32]:
DATA_PRODUCT_DIR = os.path.join(parent_dir,'timestamp_xrayDB.db')
DATA_PRODUCT_DIR

'/home/padialjr/ALEXIS_pipeline/timestamp_xrayDB.db'